# Chrono Phantasma
### Resolucion del problema de _load balance_ usando _Game Theory_ y _Genetic Algorithms_

### El problema

Distribuir trabajos de varios clientes entre varias computadoras distribuidas, intentando minimizar el tiempo de respuesta de todos

#### Estatico vs Dinamico 

Las estrategias estaticas no agregan un overhead para determinar donde distribuir los trabajos. A su vez requieren conocer a priori informacion del sistema:

* Round Robin
* Control Centralizado
* Random

Este tipo de estrategias varian mucho en su efectividad segun las cualidades de el/los clientes y de la de las maquinas. Asimismo, no pueden reaccionar ante sobrecargas en las maquinas. 

Los algoritmos dinamicos para distribucion de trabajos tienen la capacidad de adaptarse a los cambios del sistema, pero requieren poder constantemente obtener informacion de las maqinas.

* Indices de carga

### Condiciones del trabajo
Supongamos que estamos modelizando un sistema distribuido que se encarga de tomar fotos del espacio y procesarlas. Hay n clientes que toman fotos de distintas partes del cosmos y m maquinas que pueden procesarlas para descubrir nuevas estrellas.

* Cada cliente i, saca una foto del espacio con una probabilidad exp( L_i )
* Cada foto puede ser infinitamente divisible 
* Cada division de la imagen puede ser analizada de forma aislada
* Cada maquina atiende los trabajos en orden de llegada
* Los trabajos llegan a cada maquina segun un proceso de Poisson 
* Los trabajos son fotos o partes de fotos a procesar
* Las fotos generadas por cada cliente son iguales en tamaño
* Cada maquina tiene una velocidad distinta para procesar una imagen completa
* El tiempo de ejecucion de un trabajo es proporcional al tamaño de la imagen enviada
* Existe capacidad suficiente en **el total maquinas** para procesar todo

Ademas se establecen las siguientes restricciones para garantizar que el problema tenga solucion
<img src="./resources/Selección_044.jpg?2">


## Partes del trabajo

### Simulador
Se desarrollo un simulador en Python basado en procesos y pipes (usados como colas de mensajes). Se divide en 3 partes:
* Clientes: Crean trabajos cada exp(L)
* Maquinas: Simulan procesar los trabajos que entran por la cola, con una velocidad x
* Simulador: Crea la simulacion en base a un archivo json que contiene las especificaciones del sistema

### Optimizador
Es un modulo escrito tambien en Python que optimiza la distribucion de trabajos de cada cliente, basandose en un algoritmo genetico.

## Ejemplo de aplicacion
### La peor idea del mundo, todos a la misma maquina

In [4]:
head ./src/worst_idea_worst.json -n 9

{
  "machines_times": [0.03, 0.03, 0.01, 0.02, 0.03, 0.05],
  "clients": [
    {
      "lambda": 3,
      "allocation": [1, 0, 0, 0, 0, 0]
    }, {
      "lambda": 3,
      "allocation": [1, 0, 0, 0, 0, 0]


In [ ]:
python3 ./src/simulation.py ./src/worst_idea_worst.json 20 ./resources/worst_idea.png

<img src="./resources/worst_idea.png?2">

## Modelando el problema segun Game Theory

El problema de _load balancing_ se puede modelar como un juego entre los n clientes que hacen las veces de jugadores. El modelo adoptado para este proyecto tiene las siguientes caracteristicas:

* Juego no cooperativo
* Juego no simetrico
  * Cualquier problema de _load balancing_ que cuente con un grupo de clientes no heterogeneos es un juego no simetrico
* Juego con estrategias mixtas
* Existe un equilibrio de Nash
* No es de suma cero

### Por que no cooperativo?

Los diversos agentes funcionando de manera egoista, buscan minimizar su propio tiempo de respuesta. Estas acciones no solo benefician al jugador, sino que benefician al resto de los jugadores.

## Movemos un solo cliente de maquina

In [5]:
head ./src/worst_idea_not_worst.json -n 9

{
  "machines_times": [0.03, 0.03, 0.01, 0.02, 0.03, 0.05],
  "clients": [
    {
      "lambda": 3,
      "allocation": [0, 1, 0, 0, 0, 0]
    }, {
      "lambda": 3,
      "allocation": [1, 0, 0, 0, 0, 0]


In [ ]:
python3 ./src/simulation.py ./src/worst_idea_not_worst.json 20 ./resources/not_the_worst_idea.png

<img src="./resources/not_the_worst_idea.png?2">

## Especificaciones del algoritmo genetico utilizado

### Cruza y mutacion:

De cada generacion se elige solo el 50% de la misma para pasar a la siguiente. El resto de la poblacion de la siguiente generacion se obtiene por cruza, mutacion o generacion espontanea, con distintas probabilidades

#### Metodo de cruza:

Para las cruzas se utiliza cruza simple, que no es mas que generar 2 nuevos individuos con la mitad de los cromosomas de los padres. Para este caso particular, es necesario normalizar las partes intercambiadas, de forma que las cruzas sean soluciones validas.

#### Metodo de mutacion:

Para las mutaciones hay 3 estrategias con distintas probabilidaes:
* Shuffle: Se mueven todos los valores del cromosoma de forma completamente aleatoria
* Swap: Se intercambian solo dos valores del cromosoma
* Realloc: Se reduce una porcion de uno de los valores del cromosoma y se lo pasa a otro

#### Metodo de generacion espontanea:

Para prevenir que las mutaciones o cruzas converjan a un minimo local, se crean individuos completamente aleatorios

### Metodo de seleccion:
Para elegir los individuos mas aptos para que pasen a la siguiente generacion y para elegir los individuos para cruzar/mutar, se utiliza tournamet selection. Para la siguiente generacion se hace torneo de a 2 y para las cruzas torneo de a 5. Los individuos de entre los cuales se elige para mutar, son aquellos que son aptos para pasar a la siguiente generacion.

El "ganador" del torneo se elije comparando las funciones de fitness. En este caso, a menor valor de fitness, mas apto es el individuo.

### Funcion de fitness
Como funcion de fitness para cada estrategia se utiliza la prediccion del tiempo que insumiria dicha estrategia en las condiciones actuales del sistema. La formula es:

<img src="./resources/Selección_042.jpg?2">
Donde 
<img src="./resources/Selección_043.jpg?2">

Ski es la fraccion de trabajo que el cliente k asigna a la maquina i, y es el L asociado al cliente K

## Mejorando la estrategia mediante GA

In [ ]:
python3 ./src/genetic_trainer.py ./src/worst_idea_worst.json ./src/worst_idea_optimization.json

In [7]:
head ./src/worst_idea_optimization.json -n 9

{
  "machines_times": [0.03, 0.03, 0.01, 0.02, 0.03, 0.05],
  "clients": [
    {
      "lambda": 3,
      "allocation": [
        0.018603551143252607, 0.06801617171946417, 0.7632661600185777, 0.12954328492496697, 0.012724835103456986,
        0.007845997090281512
      ]


In [ ]:
python3 ./src/simulation.py ./src/worst_idea_optimization.json 20 ./resources/optimized_the_worst_idea.pnghead ./src/worst_idea_not_worst.json -n 9

<img src="./resources/optimized_the_worst_idea.png?2">

# Material

### Base para el trabajo
* https://www.researchgate.net/publication/292394898_A_Combination_of_Game_Theory_and_Genetic_Algorithm_for_Load_Balancing_in_Distributed_Computer_Systems_PLEASE_REFERENCE_IN_YOUR_PAPERS

### Load balance
* https://www.sciencedirect.com/science/article/pii/S1877042814028705 

### Game Theory & Load balance 
* https://we.vub.ac.be/sites/default/files/files/bachelor_scriptie_filip_moons.pdf
* http://citeseerx.ist.psu.edu/viewdoc/download?doi=10.1.1.121.6142&rep=rep1&type=pdf

### Problemas con Genetic Algorithms
* https://iccl.inf.tu-dresden.de/w/images/b/b7/GA_for_TSP.pdf (Problema del viajante)
* http://www.iaeng.org/publication/WCECS2012/WCECS2012_pp363-368.pdf (Generacion de identikits)
* https://pdfs.semanticscholar.org/05a4/640bda446afac4466cff792de6a19e36ba47.pdf (Generacion de prendas)